In [1]:
from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    Lambda,
    Resize,
    ScaleIntensityRange,
    SpatialCrop,
    ToTensor,
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import monai.utils as utils

import torch
import tempfile
import shutil
import os
import sys

import numpy as np
import skimage

from scipy.stats import multivariate_normal
from scipy.stats import mode

import ffmpeg
import av

import itk
from itk import TubeTK as ttk

from time import perf_counter
from contextlib import contextmanager

from ARGUSUtils import *
from ARGUSUtils_IO import *
from ARGUSUtils_Linearization import *
from ARGUSUtils_Transforms import *
from ARGUSUtils_ARNet import *

C:\Users\stephen.aylward\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


AttributeError: module 'itk' has no attribute 'itkARGUS'

In [ ]:
filename = "../Data/Final15/BAMC-PTXNoSliding/Do_Not_Use/219ns_image_1895283541879_clean.mov"
#filename = "../Data/Final15/BAMC-PTXSliding/212s_image_128692595484031_CLEAN.mov"

In [ ]:
height,width = shape_video(filename)

with time_this("Load Time:"):
    us_video = load_video(filename,height,width)
with time_this("Linearization Time:"):
    us_video_linear = linearize_video(us_video).transpose([2,1,0])

In [ ]:
device = torch.device("cpu")

In [ ]:
arnet_model_filename = "./Models/BAMC_PTX_ARUNET-3D-PR-Final15/best_model.vfold_0.pth"

with time_this("Load ARNet model"):
    arnet_model = arnet_load_model(arnet_model_filename)

In [ ]:
with time_this("Preprocess for ARNet"):
    arnet_input_tensor = arnet_preprocess_image(us_video_linear)

itk.imwrite(itk.GetImageFromArray(arnet_input_array[0,0].astype(np.float32)), "ARUNet_preprocessed_input.mha")

In [ ]:
with time_this("ARNet Inference Time:"):
    arnet_output = arnet_inference(arnet_input_tensor, arnet_model, device)
    
itk.imwrite(arnet_output, "ARUNet_output.mha")

In [ ]:
with time_this("CPU ROI Extraction Time:"):
    roinet_input_roi = roinet_segment_roi(arnet_input_tensor, arnet_output)

itk.imwrite(itk.GetImageFromArray(roinet_input_roi[0,0]), "ROINet_input_roi.mha")

In [ ]:
roinet_model_filename = "./Models/BAMC_PTX_ROINet-StdDevExtended-ExtrudedNS-Final15/best_model.vfold_0.pth"

with time_this("Load ROINet model"):
    roinet_model = roinet_load_model(roinet_model_filename)

In [ ]:
with time_this("Preprocess for ROINet"):
    roinet_input_tensor = roinet_preprocess_image(roinet_input_roi)

itk.imwrite(itk.GetImageFromArray(roinet_input_array[0,0].astype(np.float32)), "ROINet_preprocessed_input.mha")

In [ ]:
with time_this("ROINet Inference Time:"):
    decision, not_sliding_count, sliding_count = roinet_inference(roinet_input_tensor, roinet_model, device, True)

print(decision)